In [106]:
import pandas as pd
import numpy as np
import os
import cv2
import easyocr
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
import io
import PIL.Image
if not hasattr(PIL.Image, 'Resampling'):
    PIL.Image.Resampling = PIL.Image
from PIL import Image


In [107]:
# image directory
image_dir = '../images/aculei-images/'

In [108]:
# reading the dataframe
df = pd.read_csv('../datasets/ocr.csv', index_col=0)
df_ocr = df.copy()

In [109]:
df_ocr.head()

,hash,image_name,camera,date_time,date,time,moon,temp
0,00100018fdffffff,TF_ACULEI_8040_DSCF0129.jpg,CAM_1,2021-07-22 23:04:07,2021-07-22,23:04:07,Full Moon,NaN
1,000000b8fcfcffff,TF_ACULEI_900_DSCF0756.jpg,CAM_1,2021-05-27 22:01:44,2021-05-27,22:01:44,Full Moon,16°C
2,0000001000787f7f,TF_ACULEI_15294_DSCF0133.jpg,CAME,2023-03-30 20:35:58,2023-03-30,20:35:58,First Quarter,9°C
3,0000001e1e3cfe7f,TF_ACULEI_11374_DSCF0064.jpg,CAM_6,2022-06-09 21:29:59,2022-06-09,21:29:59,Waxing Gibbous,15°C
4,00080818fcfeffff,TF_ACULEI_4106_DSCF4336.jpg,CAM_1,2021-06-16 23:41:09,2021-06-16,23:41:09,First Quarter,19°C


In [110]:
reader = easyocr.Reader(['en'], gpu=True) 

In [111]:
def crop_bottom_third_and_display(image_path):
    # Open and display the original image
    img = Image.open(image_path)

    # get image dimensions
    width, height = img.size
    
    # define the coordinates for cropping
    left = 0
    upper = 0
    right = width
    lower = int(height * 17 / 18)  # keep only the bottom 1/3 of the image
    
    # Crop the bottom third of the image
    cropped_img = img.crop((left, lower, right, height))
    output = io.BytesIO()
    cropped_img.save(output, format='JPEG')  # Change format as needed (JPEG, PNG, etc.)
    cropped_img_bytes = output.getvalue()

    # Display the cropped image
    cropped_img = Image.open(io.BytesIO(cropped_img_bytes))
    
    return cropped_img_bytes

In [112]:
for index, row in tqdm(df_ocr.iterrows(), total=len(df_ocr)):
    image_path = os.path.join(image_dir, row['image_name'])
    if os.path.isdir(image_path) and image_path.startswith("."):
        continue 
    if not pd.isnull(row['temp']):
        continue
    resized_image_bytes = crop_bottom_third_and_display(image_path)
    resize_image = Image.open(io.BytesIO(resized_image_bytes))
    text = reader.readtext(resized_image_bytes)
    text = ' '.join(' '.join(map(str, t)) for t in text)    
    
    # saving the temp always
    celsius_pattern = r"-?\s*\d+\s*\*"
    celsius_matches = re.findall(celsius_pattern, text)
    df_ocr.loc[index, 'temp'] = celsius_matches[0] if celsius_matches else None


  0%|          | 0/16874 [00:00<?, ?it/s]

  2%|▏         | 300/16874 [01:06<1:00:59,  4.53it/s]


KeyboardInterrupt: 

In [ ]:
csv_path = '../datasets/ocr.csv'
df_ocr.to_csv(csv_path)